## Created by <a href="https://github.com/yunsuxiaozi">yunsuxiaozi</a>  2024/7/31

####  在<a href="https://www.kaggle.com/code/yunsuxiaozi/chatglm6b-huanhuan-finetune-training">chatglm6b-huanhuan-finetune(training)</a>这个notebook里,我们使用了甄嬛传的数据集完成了chatglm-6b大模型的微调,训练出了专属于我们的个性化AI--chat_huanhuan。如果有人想要体验chat_huanhuan,可以使用这个notebook来进行体验,下面为示例代码。

#### 注:由于这里的代码都在模型微调的代码里出现过,如果有人想看代码的具体解释可以看模型微调的notebook.

In [1]:
!git clone https://github.com/THUDM/ChatGLM-6B.git

Cloning into 'ChatGLM-6B'...
remote: Enumerating objects: 1252, done.
remote: Counting objects: 100% (17/17), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 1252 (delta 8), reused 11 (delta 6), pack-reused 1235
Receiving objects: 100% (1252/1252), 9.15 MiB | 17.78 MiB/s, done.
Resolving deltas: 100% (737/737), done.


In [2]:
!pip install -r ChatGLM-6B/requirements.txt  #安装chatglm需要依赖的库

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 59.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 416.6/416.6 kB 26.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 79.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.2/318.2 kB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 50.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 61.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.4/121.4 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.7/73.7 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 11.6 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully uninstalled tokenizers-0.19.1
  Attempting uninstall: websockets


In [3]:
!pip install -q rouge_chinese nltk jieba datasets 

In [4]:
!git clone https://huggingface.co/THUDM/chatglm-6b-int4

Cloning into 'chatglm-6b-int4'...
remote: Enumerating objects: 137, done.
remote: Total 137 (delta 0), reused 0 (delta 0), pack-reused 137 (from 1)
Receiving objects: 100% (137/137), 62.10 KiB | 15.53 MiB/s, done.
Resolving deltas: 100% (79/79), done.
Filtering content: 100% (2/2), 3.62 GiB | 19.37 MiB/s, done.


In [5]:
# AutoTokenizer自动加载与模型对应的分词器,AutoModel自动加载预训练模型
from transformers import AutoTokenizer, AutoModel

model_path = "chatglm-6b-int4"#模型的参数
#根据模型的路径加载预训练分词器,允许远程加载代码(trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)
#根据模型的路径加载预训练模型,允许远程加载代码(trust_remote_code=True),half是半精度浮点数,cuda是移动到GPU上
model = AutoModel.from_pretrained(model_path, trust_remote_code=True).half().cuda()

Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.
Explicitly passing a `revision` is encouraged when loading a configuration with custom code to ensure no malicious code has been contributed in a newer revision.
Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.


No compiled kernel found.
Compiling kernels : /root/.cache/huggingface/modules/transformers_modules/chatglm-6b-int4/quantization_kernels_parallel.c
Compiling gcc -O3 -fPIC -pthread -fopenmp -std=c99 /root/.cache/huggingface/modules/transformers_modules/chatglm-6b-int4/quantization_kernels_parallel.c -shared -o /root/.cache/huggingface/modules/transformers_modules/chatglm-6b-int4/quantization_kernels_parallel.so
Load kernel : /root/.cache/huggingface/modules/transformers_modules/chatglm-6b-int4/quantization_kernels_parallel.so
Setting CPU quantization kernel threads to 2
Parallel kernel is not recommended when parallel num < 4.
Using quantization cache
Applying quantization to glm layers


In [6]:
import torch#pytorch这个深度学习框架
from transformers import AutoConfig#自动下载和配置预训练模型的配置

#根据模型路径加载config,允许远程加载代码(trust_remote_code=True),大模型输入序列的最大长度
config = AutoConfig.from_pretrained(model_path, trust_remote_code=True, pre_seq_len=256)
#根据模型的路径和参数加载模型,允许远程加载代码(trust_remote_code=True)
model = AutoModel.from_pretrained(model_path, config=config, trust_remote_code=True)

#从二进制(binary)文件中加载模型的状态字典,这个参数字典一般是在某个检查点(checkpoint)保存下来的。
prefix_state_dict = torch.load("/kaggle/input/chatglm6b-huanhuan-finetune-training/output/infer-chatglm-6b-int4-pt-256-5e-2/checkpoint-500/pytorch_model.bin")
#进行参数的更新
new_prefix_state_dict = {}
for k, v in prefix_state_dict.items():
    new_prefix_state_dict[k[len("transformer.prefix_encoder."):]] = v
model.transformer.prefix_encoder.load_state_dict(new_prefix_state_dict)

#half是半精度浮点数,cuda是移动到GPU上
model = model.half().cuda()
#将模型prefix_encoder部分的参数换成全精度浮点数float32
model.transformer.prefix_encoder.float()
#将大模型换成评估模式
model = model.eval()

Explicitly passing a `revision` is encouraged when loading a configuration with custom code to ensure no malicious code has been contributed in a newer revision.
Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.


No compiled kernel found.
Compiling kernels : /root/.cache/huggingface/modules/transformers_modules/chatglm-6b-int4/quantization_kernels_parallel.c
Compiling gcc -O3 -fPIC -pthread -fopenmp -std=c99 /root/.cache/huggingface/modules/transformers_modules/chatglm-6b-int4/quantization_kernels_parallel.c -shared -o /root/.cache/huggingface/modules/transformers_modules/chatglm-6b-int4/quantization_kernels_parallel.so
Load kernel : /root/.cache/huggingface/modules/transformers_modules/chatglm-6b-int4/quantization_kernels_parallel.so
Setting CPU quantization kernel threads to 2
Parallel kernel is not recommended when parallel num < 4.
Using quantization cache
Applying quantization to glm layers


Some weights of ChatGLMForConditionalGeneration were not initialized from the model checkpoint at chatglm-6b-int4 and are newly initialized: ['transformer.prefix_encoder.embedding.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
question='朕的后宫佳丽三千,朕最喜欢的就是你。'
response, history = model.chat(tokenizer, question, history=[])
print(f"question:{question}\nresponse:{response}")

2024-07-31 13:56:00.003249: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-31 13:56:00.003347: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-31 13:56:00.131327: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


question:朕的后宫佳丽三千,朕最喜欢的就是你。
response:皇上最喜我，我自然高兴。
